In [3]:
from bs4 import BeautifulSoup #スクレイピングのためのライブラリ
import requests #HTTP操作用
import re
import time
import pandas as pd
import numpy as np
import sqlite3

/Users/takahashikaisei/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [92]:
y = 2024
m = 1
url = f'https://www.data.jma.go.jp/obd/stats/etrn/view/daily_a1.php?prec_no=44&block_no=0370&year={y}&month={m}'
r = requests.get(url)

In [93]:
r

<Response [200]>

In [94]:
soup = BeautifulSoup(r.content, 'html.parser')
print(type(soup))

<class 'bs4.BeautifulSoup'>


In [95]:
print(soup.find('title').string)

気象庁｜過去の気象データ検索


classをdata_0_0で絞ればとりたいものはとれる

In [96]:
tags2 = soup.find('td',class_ = 'data_0_0').string
tags2

'0.0'

これをfor文で回して、データベースにいい感じに格納すればいい

for文で全部を取得できるようにする

In [97]:
tags3 = soup.find_all('td',class_ = 'data_0_0')

data = []
for i in tags3:
    data.append(i.string)


In [98]:
data

['0.0',
 '0.0',
 '0.0',
 '9.7',
 '12.4',
 '5.9',
 '///',
 '///',
 '4.0',
 '8.4',
 '北北西',
 '16.6',
 '北西',
 '北北西',
 '9.1',
 '///',
 '///',
 '0.0',
 '0.0',
 '0.0',
 '6.5',
 '8.9',
 '3.6',
 '///',
 '///',
 '2.0',
 '4.0',
 '東南東',
 '6.8',
 '北北西',
 '北',
 '2.1',
 '///',
 '///',
 '0.0',
 '0.0',
 '0.0',
 '8.0',
 '11.2',
 '3.3',
 '///',
 '///',
 '1.5',
 '3.1',
 '北',
 '5.3',
 '北北西',
 '北',
 '0.4',
 '///',
 '///',
 '0.0',
 '0.0',
 '0.0',
 '10.0',
 '14.9',
 '6.5',
 '///',
 '///',
 '3.4',
 '7.8',
 '北西',
 '15.1',
 '北西',
 '北',
 '6.8',
 '///',
 '///',
 '0.0',
 '0.0',
 '0.0',
 '7.9',
 '13.1',
 '3.1',
 '///',
 '///',
 '1.8',
 '3.8',
 '南西',
 '5.2',
 '南西',
 '北北西',
 '8.5',
 '///',
 '///',
 '0.0',
 '0.0',
 '0.0',
 '9.2',
 '16.0',
 '3.1',
 '///',
 '///',
 '1.3',
 '3.6',
 '南西',
 '4.8',
 '西北西',
 '北北東',
 '8.7',
 '///',
 '///',
 '0.0',
 '0.0',
 '0.0',
 '8.6',
 '11.5',
 '5.4',
 '///',
 '///',
 '2.8',
 '6.7',
 '北西',
 '12.1',
 '北北西',
 '北',
 '3.2',
 '///',
 '///',
 '0.0',
 '0.0',
 '0.0',
 '5.7',
 '9.6',
 '1.8',
 '///',

In [99]:
def day_num_count(y, m):
    if m in [1, 3, 5, 7, 8, 10, 12]:
        day_num = 31
    elif m in [4, 6, 9, 11]:
        day_num = 30
    elif m == 2:    
        if y % 400 == 0:
            day_num = 29
        elif y % 100 == 0:
            day_num = 28
        elif y % 4 == 0:
            day_num = 29
        else:
            day_num = 28
    
    return day_num

リストに格納は嫌だからデータフレームに格納したい

In [100]:
tags3 = soup.find_all('td', class_='data_0_0')

data = []
for i in tags3:
    data.append(i.string)

day_num_count(y, m)

final_list = []
final_list = np.array_split(data, day_num) #31日だから31ずつに区切ればいい

# データフレームに変換する
df = pd.DataFrame(final_list, columns=[
    "合計降水量",
    "最大一時間降水量",
    "最大10分間降水量",
    "平均気温",
    "最高気温",
    "最低気温",
    "平均湿度",
    "最小湿度",
    "平均風速",
    "最大風速",
    "最大風速の風向き",
    "最大瞬間風速",
    "最大瞬間風速の風向き",
    "最多風向き",
    "日照時間",
    "降雪の深さの合計",
    "最深積雪"
    ])

df

,合計降水量,最大一時間降水量,最大10分間降水量,平均気温,最高気温,最低気温,平均湿度,最小湿度,平均風速,最大風速,最大風速の風向き,最大瞬間風速,最大瞬間風速の風向き,最多風向き,日照時間,降雪の深さの合計,最深積雪
0,0.0,0.0,0.0,9.7,12.4,5.9,///,///,4.0,8.4,北北西,16.6,北西,北北西,9.1,///,///
1,0.0,0.0,0.0,6.5,8.9,3.6,///,///,2.0,4.0,東南東,6.8,北北西,北,2.1,///,///
2,0.0,0.0,0.0,8.0,11.2,3.3,///,///,1.5,3.1,北,5.3,北北西,北,0.4,///,///
3,0.0,0.0,0.0,10.0,14.9,6.5,///,///,3.4,7.8,北西,15.1,北西,北,6.8,///,///
4,0.0,0.0,0.0,7.9,13.1,3.1,///,///,1.8,3.8,南西,5.2,南西,北北西,8.5,///,///
5,0.0,0.0,0.0,9.2,16.0,3.1,///,///,1.3,3.6,南西,4.8,西北西,北北東,8.7,///,///
6,0.0,0.0,0.0,8.6,11.5,5.4,///,///,2.8,6.7,北西,12.1,北北西,北,3.2,///,///
7,0.0,0.0,0.0,5.7,9.6,1.8,///,///,3.3,6.4,北西,11.9,北西,北北西,9.2,///,///
8,0.0,0.0,0.0,5.7,10.1,1.9,///,///,1.9,3.6,北,7.0,北,北,9.1,///,///
9,0.0,0.0,0.0,7.6,12.2,1.3,///,///,1.9,5.3,北北東,10.7,北,北,5.8,///,///


In [101]:
day = []
for i in range(1, day_num+1):
    day.append(i)

hiduke = []
for tasu in day:
    hiduke.append(str(m) + '/' + str(tasu))

s = pd.Series(hiduke, name='date')

df.insert(0, '日付', s)

リストに格納できるのはできる
for文の回し方はわかったから次はデータベースへの格納を考えよう

データベースへの格納を考えるよ
調べたら意外と簡単に変換できた。

In [103]:
con = sqlite3.connect('weather_scrape.sqlite') #データベースにアクセス

df.to_sql('weather_scrape_data', con, if_exists='replace', index=False) #データベースに変換、if_exists='replace'で同じ名前のものがあったときに置き換える。index=Falseでindexを入れないように指定してる

con.close() #データベースを閉じる

In [104]:
con = sqlite3.connect('weather_scrape.sqlite') #データベースにアクセス
c = con.cursor()

c.execute('SELECT * FROM weather_scrape_data')
rows = c.fetchall()

# 結果を表示
for row in rows:
    print(row)

# データベースを閉じる
con.close()

('1/1', '0.0', '0.0', '0.0', '9.7', '12.4', '5.9', '///', '///', '4.0', '8.4', '北北西', '16.6', '北西', '北北西', '9.1', '///', '///')
('1/2', '0.0', '0.0', '0.0', '6.5', '8.9', '3.6', '///', '///', '2.0', '4.0', '東南東', '6.8', '北北西', '北', '2.1', '///', '///')
('1/3', '0.0', '0.0', '0.0', '8.0', '11.2', '3.3', '///', '///', '1.5', '3.1', '北', '5.3', '北北西', '北', '0.4', '///', '///')
('1/4', '0.0', '0.0', '0.0', '10.0', '14.9', '6.5', '///', '///', '3.4', '7.8', '北西', '15.1', '北西', '北', '6.8', '///', '///')
('1/5', '0.0', '0.0', '0.0', '7.9', '13.1', '3.1', '///', '///', '1.8', '3.8', '南西', '5.2', '南西', '北北西', '8.5', '///', '///')
('1/6', '0.0', '0.0', '0.0', '9.2', '16.0', '3.1', '///', '///', '1.3', '3.6', '南西', '4.8', '西北西', '北北東', '8.7', '///', '///')
('1/7', '0.0', '0.0', '0.0', '8.6', '11.5', '5.4', '///', '///', '2.8', '6.7', '北西', '12.1', '北北西', '北', '3.2', '///', '///')
('1/8', '0.0', '0.0', '0.0', '5.7', '9.6', '1.8', '///', '///', '3.3', '6.4', '北西', '11.9', '北西', '北北西', '9.2', '///',

とってないデータもちゃんと表示するからよくない＋データベースを完全に置き換えてるから前取ったのが消えてる。追加式にしないとだめ

さっき作った余計なデータベースを削除したコード、一応コメントアウトにしとく

In [88]:
# #データベースを削除する
# con = sqlite3.connect('weather_scrape.sqlite')
# cur = con.cursor()

# cur.execute('DROP TABLE IF EXISTS weather_scrape_12month_data')
# con.commit()
# con.close()

In [107]:
tags4 = soup.find_all('tr',class_ = 'mtx')
tags4

[<tr class="mtx">
 <th rowspan="3" scope="col">日</th>
 <th colspan="3" scope="colgroup">降水量</th>
 <th colspan="3" scope="colgroup">気温</th>
 <th colspan="2" scope="colgroup">湿度</th>
 <th colspan="6" scope="colgroup">風向・風速</th>
 <th rowspan="3" scope="col">日照<br/>時間<br/>(h)</th>
 <th colspan="2" scope="colgroup">雪</th>
 </tr>,
 <tr class="mtx">
 <th rowspan="2" scope="col">合計<br/>(mm)</th><th rowspan="2" scope="col">最大1時間<br/>(mm)</th><th rowspan="2" scope="col">最大10分間<br/>(mm)</th>
 <th rowspan="2" scope="col">平均<br/>(℃)</th><th rowspan="2" scope="col">最高<br/>(℃)</th><th rowspan="2" scope="col">最低<br/>(℃)</th>
 <th rowspan="2" scope="col">平均<br/>(％)</th><th rowspan="2" scope="col">最小<br/>(％)</th>
 <th rowspan="2" scope="col">平均風速<br/>(m/s)</th><th colspan="2" scope="colgroup">最大</th><th colspan="2" scope="colgroup">最大瞬間</th><th rowspan="2" scope="col">最多<br/>風向</th>
 <th rowspan="2" scope="col">降雪の深さの<br/>合計(cm)</th><th rowspan="2" scope="col">最深積雪<br/>(cm)</th>
 </tr>,
 <tr class="mtx"

In [110]:
tags5 = soup.find('table',class_ = 'data2_s')
tags5

<table class="data2_s" id="tablefix1">
<tr class="mtx">
<th rowspan="3" scope="col">日</th>
<th colspan="3" scope="colgroup">降水量</th>
<th colspan="3" scope="colgroup">気温</th>
<th colspan="2" scope="colgroup">湿度</th>
<th colspan="6" scope="colgroup">風向・風速</th>
<th rowspan="3" scope="col">日照<br/>時間<br/>(h)</th>
<th colspan="2" scope="colgroup">雪</th>
</tr>
<tr class="mtx">
<th rowspan="2" scope="col">合計<br/>(mm)</th><th rowspan="2" scope="col">最大1時間<br/>(mm)</th><th rowspan="2" scope="col">最大10分間<br/>(mm)</th>
<th rowspan="2" scope="col">平均<br/>(℃)</th><th rowspan="2" scope="col">最高<br/>(℃)</th><th rowspan="2" scope="col">最低<br/>(℃)</th>
<th rowspan="2" scope="col">平均<br/>(％)</th><th rowspan="2" scope="col">最小<br/>(％)</th>
<th rowspan="2" scope="col">平均風速<br/>(m/s)</th><th colspan="2" scope="colgroup">最大</th><th colspan="2" scope="colgroup">最大瞬間</th><th rowspan="2" scope="col">最多<br/>風向</th>
<th rowspan="2" scope="col">降雪の深さの<br/>合計(cm)</th><th rowspan="2" scope="col">最深積雪<br/>(cm)</th>
<

In [112]:
tags6 = soup.find_all('td')
tags6

[<td><img alt="一覧表" src="../../data/icon/sellected/table.gif"/></td>,
 <td><a href="daily_a1.php?prec_no=44&amp;block_no=0370&amp;year=2024&amp;month=01&amp;day=&amp;view=g_"><img alt="グラフ" src="../../data/icon/true/graph.gif"/></a></td>,
 <td><a href="daily_a1.php?prec_no=44&amp;block_no=0370&amp;year=2024&amp;month=01&amp;day=&amp;view=p1s"><img alt="見出しの固定" src="../../data/icon/true/scroll_ari.gif"/></a></td>,
 <td><a href="../index.php?prec_no=44&amp;block_no=0370&amp;year=2024&amp;month=01&amp;day=&amp;view=p1"><img alt="メニューに戻る" src="../../data/icon/true/2menu.gif"/></a></td>,
 <td><img alt="主な要素" src="../../data/icon/sellected/default.gif"/></td>,
 <td><a href="daily_a1.php?prec_no=44&amp;block_no=0370&amp;year=2024&amp;month=01&amp;day=&amp;view=a1"><img alt="詳細（降水量）" src="../../data/icon/true/pre.gif"/></a></td>,
 <td><a href="daily_a1.php?prec_no=44&amp;block_no=0370&amp;year=2024&amp;month=01&amp;day=&amp;view=a2"><img alt="詳細（気温・蒸気圧・湿度）" src="../../data/icon/true/temvh.gif"

In [113]:
# 'table' タグの中の 'td' タグをすべて抽出
td_tags = tags5.find_all('td')

# td_tagsは<td>タグのリスト
# 必要に応じて、各<td>タグの内容を抽出したり、他の操作を行うことができます。

# 例：各<td>タグのテキストを表示
for td in td_tags:
    print(td.get_text())

1
0.0
0.0
0.0
9.7
12.4
5.9
///
///
4.0
8.4
北北西
16.6
北西
北北西
9.1
///
///
2
0.0
0.0
0.0
6.5
8.9
3.6
///
///
2.0
4.0
東南東
6.8
北北西
北
2.1
///
///
3
0.0
0.0
0.0
8.0
11.2
3.3
///
///
1.5
3.1
北
5.3
北北西
北
0.4
///
///
4
0.0
0.0
0.0
10.0
14.9
6.5
///
///
3.4
7.8
北西
15.1
北西
北
6.8
///
///
5
0.0
0.0
0.0
7.9
13.1
3.1
///
///
1.8
3.8
南西
5.2
南西
北北西
8.5
///
///
6
0.0
0.0
0.0
9.2
16.0
3.1
///
///
1.3
3.6
南西
4.8
西北西
北北東
8.7
///
///
7
0.0
0.0
0.0
8.6
11.5
5.4
///
///
2.8
6.7
北西
12.1
北北西
北
3.2
///
///
8
0.0
0.0
0.0
5.7
9.6
1.8
///
///
3.3
6.4
北西
11.9
北西
北北西
9.2
///
///
9
0.0
0.0
0.0
5.7
10.1
1.9
///
///
1.9
3.6
北
7.0
北
北
9.1
///
///
10
0.0
0.0
0.0
7.6
12.2
1.3
///
///
1.9
5.3
北北東
10.7
北
北
5.8
///
///
11
0.0
0.0
0.0
5.2
7.2
3.4
///
///
3.4
5.7
北
11.2
北
北北西
0.4
///
///
12
0.0
0.0
0.0
7.4
13.8
1.0
///
///
2.0
3.9
北北西
7.7
北
北北西
9.1
///
///
13
///
///
///
///
///
///
///
///
///
///
///
///
///
///
///
///
///
14
///
///
///
///
///
///
///
///
///
///
///
///
///
///
///
///
///
15
///
///
///
///
///
///
///
///

In [120]:
tags7 = soup.find_all('table', {'id': 'tablefix1'})
for tag7 in tags7:
    tags8 = tag7.find('tr', {'style': 'text-align:right;'})
    print(tags8)

<tr class="mtx" style="text-align:right;"><td style="white-space:nowrap"><div class="a_print"><a href="hourly_a1.php?prec_no=44&amp;block_no=0370&amp;year=2024&amp;month=01&amp;day=1&amp;view=p1">1</a></div></td><td class="data_0_0">0.0</td><td class="data_0_0">0.0</td><td class="data_0_0">0.0</td><td class="data_0_0">9.7</td><td class="data_0_0">12.4</td><td class="data_0_0">5.9</td><td class="data_0_0">///</td><td class="data_0_0">///</td><td class="data_0_0">4.0</td><td class="data_0_0">8.4</td><td class="data_0_0" style="text-align:center">北北西</td><td class="data_0_0">16.6</td><td class="data_0_0" style="text-align:center">北西</td><td class="data_0_0" style="text-align:center">北北西</td><td class="data_0_0">9.1</td><td class="data_0_0">///</td><td class="data_0_0">///</td></tr>


In [122]:
tags8

<tr class="mtx" style="text-align:right;"><td style="white-space:nowrap"><div class="a_print"><a href="hourly_a1.php?prec_no=44&amp;block_no=0370&amp;year=2024&amp;month=01&amp;day=1&amp;view=p1">1</a></div></td><td class="data_0_0">0.0</td><td class="data_0_0">0.0</td><td class="data_0_0">0.0</td><td class="data_0_0">9.7</td><td class="data_0_0">12.4</td><td class="data_0_0">5.9</td><td class="data_0_0">///</td><td class="data_0_0">///</td><td class="data_0_0">4.0</td><td class="data_0_0">8.4</td><td class="data_0_0" style="text-align:center">北北西</td><td class="data_0_0">16.6</td><td class="data_0_0" style="text-align:center">北西</td><td class="data_0_0" style="text-align:center">北北西</td><td class="data_0_0">9.1</td><td class="data_0_0">///</td><td class="data_0_0">///</td></tr>

-----

In [7]:
y = 2024
m = 1
url = f'https://www.data.jma.go.jp/obd/stats/etrn/view/daily_a1.php?prec_no=44&block_no=0370&year={y}&month={m}'
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')

In [8]:
tags7 = soup.find_all('table', {'id': 'tablefix1'}) #これでテーブルを絞る
for tag7 in tags7:
    for tttag7 in tag7.find_all('tr'):
        if tttag7.find('a'): #aタグがある場合が日数がデータが入力されてる日
            data = []
            for td in tttag7.find_all('td'):
                 text = td.get_text(strip=True)
                 data.append(text)

            
             
            
            con = sqlite3.connect('weather_scrape.sqlite')
            cur = con.cursor()

            cur.execute('INSERT INTO weather_kisyocho (日,合計降水量,最大一時間降水量,最大10分間降水量,平均気温,最高気温,最低気温,平均湿度,最小湿度,平均風速,最大風速,最大風速の風向き,最大瞬間風速,最大瞬間風速の風向き,最多風向き,日照時間,降雪の深さの合計,最深積雪) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)',
                      (str(m) + '/' + str(data[0]),
                       data[1],
                       data[2],
                       data[3],
                       data[4],
                       data[5],
                       data[6],
                       data[7],
                       data[8],
                       data[9],
                       data[10],
                       data[11],
                       data[12],
                       data[13],
                       data[14],
                       data[15],
                       data[16],
                       data[17]
                       )) # データベースにデータを挿入
            con.commit() #追加

con.close()

データベースを一覧表示したいとき

In [13]:
con = sqlite3.connect('weather_scrape.sqlite')
cur = con.cursor()

sql_select = "SELECT * FROM weather_kisyocho;"

cur.execute(sql_select)
for r in cur:
    print(r)

con.close()

OperationalError: no such table: weather_kisyocho

テーブルを作りたい時

In [170]:
con = sqlite3.connect('weather_scrape.sqlite')
cur = con.cursor()

# テーブルを作成するSQL
table_weather = 'CREATE TABLE weather_kisyocho(日 str,合計降水量 str,最大一時間降水量 str,最大10分間降水量 str,平均気温 str,最高気温 str,最低気温 str,平均湿度 str,最小湿度 str,平均風速 str,最大風速 str,最大風速の風向き str,最大瞬間風速 str,最大瞬間風速の風向き str,最多風向き str,日照時間 str,降雪の深さの合計 str,最深積雪 str);'

# 4．SQLを実行する
cur.execute(table_weather)
con.close()

テーブルを削除したい時

In [10]:
con = sqlite3.connect('weather_scrape.sqlite')
cur = con.cursor()

cur.execute('DROP TABLE IF EXISTS weather_kisyocho')
con.commit()
con.close()
